In [ ]:
import scanpy as sc
import scvi
import numpy as np

In [ ]:
import anndata as ad
from anndata import AnnData

In [ ]:
from scipy import sparse

In [ ]:
import pandas as pd

In [ ]:
rna_adata = scvi.data.read_h5ad("/omics/groups/OE0533/internal/katharina/scDoRI/gastrulation_data/jupyter_notebooks/anndata_rna.h5ad")
atac_adata = scvi.data.read_h5ad("/omics/groups/OE0533/internal/katharina/scDoRI/gastrulation_data/jupyter_notebooks/anndata_atac_peak_matrix.h5ad")

In [ ]:
#adata = scvi.data.read_10x_multiome("filtered_feature_bc_matrix")

In [ ]:
#old = scvi.data.read_h5ad("/omics/groups/OE0533/internal/katharina/scDoRI/gastrulation_data/old_data_three_timepoints/anndata_rna.h5ad")

In [ ]:
rna_adata = rna_adata[:,rna_adata.var.index.isin(old.var.index)]

In [ ]:
annotations = pd.merge(old.var, rna_adata.var, on="gene", how = "inner")
annotations.index = old.var.index

In [ ]:
annotations = annotations.rename(columns={"Accession":"ID", "End":"end", "Start":"start", "Chromosome":"chr"})

In [ ]:
annotations = annotations.drop(["gene", "Strand"], axis = 1)


In [ ]:
rna_adata.var = annotations

In [ ]:
matrix = sparse.hstack((rna_adata.X, atac_adata.X))

In [ ]:
rna_df = rna_adata.var
#rna_df = rna_df.rename(columns = {"gene":"ID"})
rna_df["modality"] = ["Gene_expression" for i in range(len(rna_df))]
rna_df

In [ ]:
atac_df = atac_adata.var
atac_df = atac_df.rename(columns={"idx":"ID"})
atac_df["modality"]=["Accessibility" for i in range(len(atac_df))]

In [ ]:
atac_df = atac_df.drop(["score"], axis = 1)

In [ ]:
df = rna_df.append(atac_df)

In [ ]:
multiome = AnnData(
X = matrix,
obs = atac_adata.obs,
var = df,
uns = rna_adata.uns)

In [ ]:
#multiome_mvi = scvi.data.organize_multiome_anndatas(multiome)

In [ ]:
#multiome_mvi.var

In [ ]:
#multiome_mvi = multiome_mvi[:, multiome_mvi.var["modality"].argsort()].copy()

In [ ]:
#multiome_mvi.var

Filter features which appear in fewer than 1% of cells.

The main batch key should correspond to the modality of the cells. All other batch annotations are added via categorical_covariate_keys.

In [ ]:
multiome_mvi

In [ ]:
scvi.model.MULTIVI.setup_anndata(multiome)#,
                                # batch_key = "modality")
                                 #categorical_covariate_keys=["sample"])

We need to specify how many features are genes and how many are genomic regions. 

In [ ]:
mvi = scvi.model.MULTIVI(
    multiome,
    n_genes=(adata.var['modality']=='Gene_expression').sum(),
    n_regions=(adata.var['modality']=='Peaks').sum(),
    dropout_rate=0.2,
    n_layers_encoder=2,
    n_layers_decoder=2,
    n_latent=20
)

arches_params = dict(
    n_genes=(multiome.var["modality"]=="Gene_expression").sum(),
    n_regions=(multiome.var["modality"]=="Peaks").sum(),
    #dropout_rate=0.2,
    #n_layers_encoder=2,
    #n_layers_decoder=2,
    #n_latent=20,
    #latent_distribution='normal'
)

mvi = scvi.model.MULTIVI(
    multiome,
    **arches_params
)
mvi.view_anndata_setup()

In [ ]:
mvi.train()